In [1]:
# Block 1: Load trained model from Notebook 1
import joblib
import pandas as pd
import numpy as np
import ee
import os
from dateutil.relativedelta import relativedelta
import datetime

# Load your 92% accurate model
rf = joblib.load('flood_model.pkl')
feature_columns = ['elevation', 'rainfall', 'slope', 'aspect']  # Update from your X.columns!

print("✅ Model loaded (92% accuracy)")
print("Expected features:", feature_columns)


✅ Model loaded (92% accuracy)
Expected features: ['elevation', 'rainfall', 'slope', 'aspect']


In [4]:
# TEMP BLOCK 1.5: Local testing only (DELETE before Git commit)
import os
os.environ['GEE_SERVICE_ACCOUNT'] = 'kshitiz-singh-286@aqualert-474513.iam.gserviceaccount.com'
os.environ['GEE_KEY_PATH'] = r'C:\Users\KSHITIZ\OneDrive\Desktop\geo_spatial_app\ML_Model\aqualert-474513-e2ed1bce4655.json'

print("✅ Temp env vars set")


✅ Temp env vars set


In [5]:
# Block 2: GEE setup + data fetch functions (secret-safe)
import ee
import os
import datetime
from dateutil.relativedelta import relativedelta

# GEE auth (reviewers: set your own env vars)
SERVICE_ACCOUNT_EMAIL = os.getenv('GEE_SERVICE_ACCOUNT')  # e.g. 'your-service@project.iam.gserviceaccount.com'
KEY_FILE_PATH = os.getenv('GEE_KEY_PATH')                 # Path to your JSON key

if SERVICE_ACCOUNT_EMAIL and KEY_FILE_PATH:
    credentials = ee.ServiceAccountCredentials(SERVICE_ACCOUNT_EMAIL, KEY_FILE_PATH)
    ee.Initialize(credentials=credentials)
    print("✅ GEE Initialized")
else:
    print("⚠️  Set env vars: GEE_SERVICE_ACCOUNT, GEE_KEY_PATH")
    ee.Initialize()  # Fallback for testing

def get_monthly_precipitation(lat, lon, year, month, max_fallback_months=12):
    """Fetch monthly rainfall (mm) from CHIRPS with fallback to prior months."""
    point = ee.Geometry.Point(lon, lat)
    target_date = datetime.datetime(year, month, 1)

    for i in range(max_fallback_months + 1):
        current_date = target_date - relativedelta(months=i)
        current_year, current_month = current_date.year, current_date.month
        source_date = current_date.strftime("%Y-%m")

        try:
            collection = (ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")
                         .filter(ee.Filter.calendarRange(current_year, current_year, 'year'))
                         .filter(ee.Filter.calendarRange(current_month, current_month, 'month'))
                         .filterBounds(point))

            if collection.size().getInfo() == 0:
                continue
            
            precip = (collection.sum()
                     .reduceRegion(ee.Reducer.first(), point, scale=5000)
                     .get('precipitation').getInfo())

            if precip is not None:
                if i > 0:
                    print(f"Used fallback: {source_date}")
                return precip
                
        except Exception:
            continue
    
    print("❌ No rainfall data found")
    return None

def get_elevation_slope_aspect(lat, lon):
    """Fetch terrain data (elevation, slope, aspect)."""
    point = ee.Geometry.Point(lon, lat)
    try:
        dem = ee.Image("USGS/SRTMGL1_003")
        terrain = ee.Terrain.products(dem)
        data = dem.addBands(terrain.select(['slope', 'aspect'])).reduceRegion(
            ee.Reducer.first(), point, scale=30).getInfo()
        
        return (data.get('elevation'), 
                data.get('slope'), 
                data.get('aspect'))
    except Exception as e:
        print(f"Terrain error: {e}")
        return None, None, None


✅ GEE Initialized


In [6]:
# Block 3: End-to-end prediction (GEE data → 92% RF model)
def predict_flood_probability(lat, lon, year, month):
    """Live flood prediction: GEE terrain/rainfall → RF model → probability."""
    
    print(f"🔄 Predicting for Haridwar ({lat}°N, {lon}°E, {year}-{month:02d})...")
    
    # GEE: Fetch live environmental data
    rainfall = get_monthly_precipitation(lat, lon, year, month)
    elevation, slope, aspect = get_elevation_slope_aspect(lat, lon)
    
    if rainfall is None:
        return None, "No rainfall data available"
    
    # Cyclical month encoding
    month_sin = np.sin(2 * np.pi * month / 12)
    month_cos = np.cos(2 * np.pi * month / 12)
    
    # Match training features exactly
    new_data = pd.DataFrame([{
        'elevation': elevation,
        'rainfall': rainfall,
        'slope': slope,
        'aspect': aspect
    }])[rf.feature_names_in_]  # Auto-matches training columns!
    
    # RF prediction (92% accuracy model)
    prob = rf.predict_proba(new_data)[0, 1]
    
    risk = "🚨 HIGH" if prob > 0.5 else "✅ LOW"
    return prob, f"{prob:.1%} ({risk})"

print("✅ Prediction pipeline ready!")


✅ Prediction pipeline ready!


In [7]:
# Block 3.5: Input validation
def validate_flood_input(lat, lon, year, month):
    """Validate Uttarakhand flood prediction inputs."""
    if not (-90 <= lat <= 90):
        raise ValueError("Latitude must be [-90, 90]")
    if not (-180 <= lon <= 180):
        raise ValueError("Longitude must be [-180, 180]")
    if not (1 <= month <= 12):
        raise ValueError("Month must be 1-12")
    if not (1990 <= year <= 2100):
        raise ValueError("Year 1990-2100 (CHIRPS data range)")
    
    print("✅ Inputs valid")
    return True

# Updated prediction with validation
def predict_flood_probability(lat, lon, year, month):
    validate_flood_input(lat, lon, year, month)
    
    print(f"🔄 Predicting for ({lat:.2f}°N, {lon:.2f}°E, {year}-{month:02d})...")
    
    # GEE data
    rainfall = get_monthly_precipitation(lat, lon, year, month)
    elevation, slope, aspect = get_elevation_slope_aspect(lat, lon)
    
    if rainfall is None:
        return None, "No rainfall data"
    
    # Model input
    new_data = pd.DataFrame([{
        'elevation': elevation or 300,  # Default if None
        'rainfall': rainfall,
        'slope': slope or 2,
        'aspect': aspect or 90
    }])[rf.feature_names_in_]
    
    prob = rf.predict_proba(new_data)[0, 1]
    risk = "🚨 HIGH RISK" if prob > 0.5 else "✅ LOW RISK"
    
    return prob, f"{prob:.1%} → {risk}"


In [19]:
# Block 4: Professional Flood Risk Assessment
print("=" * 60)
print("           UTTARAKHAND FLOOD RISK ASSESSMENT")
print("                  (Google Earth Engine + RF Model)")
print("=" * 60)

def professional_predict():
    print("\nEnter coordinates (press Enter for defaults):")
    
    lat = float(input("Latitude (30.32 Rishikesh): ") or 30.32)
    lon = float(input("Longitude (78.29 Rishikesh): ") or 78.29)
    year = int(input("Year (2013): ") or 2013)
    month = int(input("Month (7): ") or 12)
    
    print("\n" + "-"*50)
    print("FLOOD RISK RESULTS")
    print("-"*50)
    
    prob, _ = predict_flood_probability(lat, lon, year, month)
    
    if prob > 0.5:
        risk_level = "HIGH RISK"
        color = "\033[91m"  # Red
    elif prob > 0.2:
        risk_level = "MODERATE RISK" 
        color = "\033[93m"  # Yellow
    else:
        risk_level = "LOW RISK"
        color = "\033[92m"  # Green
    
    print(f"Location:    {lat:.3f}°N, {lon:.3f}°E")
    print(f"Date:        {year}-{month:02d}")
    print(f"Probability: {prob:.1%}")
    print(f"Risk Level:  {color}{risk_level}\033[0m")
    print(f"Model:       Random Forest (92% accuracy)")
    print("-"*50)

professional_predict()




           UTTARAKHAND FLOOD RISK ASSESSMENT
                  (Google Earth Engine + RF Model)

Enter coordinates (press Enter for defaults):


Latitude (30.32 Rishikesh):  
Longitude (78.29 Rishikesh):  
Year (2013):  
Month (7):  



--------------------------------------------------
FLOOD RISK RESULTS
--------------------------------------------------
✅ Inputs valid
🔄 Predicting for (30.32°N, 78.29°E, 2013-12)...
Location:    30.320°N, 78.290°E
Date:        2013-12
Probability: 4.0%
Risk Level:  LOW RISK
Model:       Random Forest (92% accuracy)
--------------------------------------------------
